In [17]:
"""
This is a notebook to perform causal ML analysis with WNV. Since classical modeling techniques were
not sufficient at predicting WNV, I want to try using causal ML to help highlight some features that
are properly tested to *cause* WNV.

NOTE: Performing classical modeling did help to highlight certain features that were important in
the prediction, but these may be based heavily on correlation, not causation. Technically, I could
also go backwards and extract the observations where the SHAP values were most significant, but 
this is a bit of a hassle, and I find causal ML to be more imformative.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [18]:
# Import the data from task 2
train_data = pd.read_pickle("../data/train.pkl")
valid_data = pd.read_pickle("../data/valid.pkl")
test_data = pd.read_pickle("../data/test.pkl")

For this, I will apply a Double Machine Learning (DML) model. This works by combining two ML models,
one to explain the target (in this case, when WNV is present), and another to explain some sort of
treatment (which would be one of the features, such as if rain was present). You compare the 
unexplained part of the target with the unexplained part of the chosen treatment, providing an 
unbiased estimate of the effect on the treatment on WNV being present.

An analogy to help make this sense is like trying to clean off fog on a lens:
- One ML model is used to clean the fog off `Y` (removing what `X`, the cofounders, explains about 
  `Y`)
- Another ML model is used to clean the fog off `T` (removing what `X` explains about `T`)
- Then, you just compare the clean part of `T` and `Y` to measure the true effect

Overall this is a nice way to remove confouding variables (variables that impact both the treatment
and the target)!

-----------

More formally, the two models are combined by calculating the residuals of both initial models, then
fitting a regression using both residuals.

$$\tilde{Y} = \theta \cdot \tilde{T} + \text{noise}$$

where $$\theta$$ is the estimated causal effect of the treatment on the outcome.

This final regression step is **where the two models come together** — by comparing what's left 
(the residuals), DML isolates the causal effect without confounding bias.

However, there are four main assumptions for applying this model:
1. Unconfoundness - All cofounders are accounting for when fitting a DML model. I'll do my best to 
   follow this assumption by including all possible features when fitting each individual DML model.
2. Overlap - No group is perfectly treated or untreated, so we can compare treated/untreated 
   observations with similar covariates. I'll check for this via AUC.
3. Consistency - There is no ambiguity in the treatment labels, which I'll ensure based on how I
   implement the models, but this is an easy assumption to uphold generally.
4. Stable Unit Treatment Value Assumption - The outcome is only affected by the treatment assigned
   to that observation, and not by treatments on other observations. I know this is true based on 
   data preprocessing from task 2.

Assumption 2 must be checked for however. This is done by estimating propensity scores, the 
probability of receiving treatment given the covariates `X`. I'll do this with logistic regression
with AUC scores, and show the columns that fit the threshold below:

- If AUC is roughly equal to 0.5, the treatment is not very predictable on X, meaning there is 
   overlap.
- AUC values greater than ~0.7 or less than ~0.3 imply that the model can (almost) perfectly predict
  treatment, suggesting poor overlap, meaning the assumption is violated.

In [ ]:
# Concatenate train_data, valid_data, and test_data into one DataFrame
# For causal ML, since there is no "prediction", there is no need to have our standard data split
df = pd.concat([train_data, valid_data, test_data], ignore_index=True)

In [ ]:
# Define outcome and features
outcome_col = 'WnvPresent'
excluded_cols = [outcome_col]
feature_cols = [col for col in df.columns if col not in excluded_cols]

# Separate out candidate treatments (binary)
candidate_treatments = [col for col in df.columns if col not in excluded_cols and
                        df[col].nunique() == 2 and set(df[col].unique()) <= {0, 1}]

# Storage for valid treatments
valid_treatments = []

# Loop over treatments
for treatment_col in candidate_treatments:
    X = df.drop(columns=[outcome_col, treatment_col])
    T = df[treatment_col].values

    # Scale features
    X_scaled = StandardScaler().fit_transform(X)

    # Fit logistic regression
    try:
        ps_model = LogisticRegression(max_iter=2000)
        ps_model.fit(X_scaled, T)
        propensity_scores = ps_model.predict_proba(X_scaled)[:, 1]

        # Compute AUC
        auc = roc_auc_score(T, propensity_scores)

        # Apply overlap threshold rule: AUC should be not too close to 1 or 0
        if 0.3 < auc < 0.7:
            valid_treatments.append((treatment_col, auc))
    except Exception as e:
        print(f"Error fitting model for {treatment_col}: {e}")

# Create DataFrame of valid treatments
valid_treatment_df = pd.DataFrame(valid_treatments, columns=["Treatment", "AUC"])
valid_treatment_df.sort_values("AUC", inplace=True)
valid_treatment_df.reset_index(drop=True, inplace=True)

     Treatment     AUC
0    ResultDir  0.4279
1      Sunrise  0.4584
2           RA  0.4733
3         Cool  0.4912
4           BR  0.5570
5           HZ  0.4399
6           TS  0.5028
7         Heat  0.5185
8  day_of_year  0.4093


These are the remaining features/variables that we can properly test individually as treatment. I'll
filter `df` to just these columns with `WnvPresent`:

In [ ]:
df = df[list(valid_treatment_df["Treatment"]) + ["WnvPresent"]]
print(df)

       WnvPresent
0               0
1               0
2               0
3               0
4               0
...           ...
10408           0
10409           0
10410           0
10411           0
10412           0

[10413 rows x 1 columns]
